
# void zero

attribute system for javascript


## towards intent

a landmark towards single token language intent based on large language models and psuedo code



### templates and attributes

most of this concept is based on two core features, attributes and code templates, this controls the generation of both of those


In [ ]:
// BASIC TEMPLATE SYSTEM, find and replace tokens
// automatically create templates out of anything marked with @Template
function template(string, object) {
	let params = Object.keys(object)
	let values = Object.values(object)
	for (let i = 0; i < params.length; i++) {
		string = (string + '').replaceAll(
			new RegExp(params[i], 'g'), values[i])
	}
	return string
}

// this is some nice distilling
// BASIC TEMPLATE SYSTEM, find and replace tokens
// Source: https://stackoverflow.com/questions/29182244/convert-a-string-to-a-template-string
function interpolate(template) {
	const names = Object.keys(template);
	const vals = Object.values(template);
  return Function.apply(null, names.concat([`return \`${this}\`;`])).apply(null, vals);
}
String.prototype.interpolate = interpolate


// ATTRIBUTE SYSTEM
const handlers = {
  function: {
    attribute: (node, [name]) => (f => handlers.function[name || f.name] = f)(node()),
    template: (node, [name]) => (f => handlers.object[name || f.name] = o => template(f.toString(), o()))(node())
  },
  object: {
    template: (node, [name]) => handlers.function[name] = f => template(f().toString(), node())
  }
}

const extractAttributeParams = line =>
  line.match(/\(([^)]*)\)/)?.[1]?.split(',').map(p => p.trim()).filter(Boolean) || [];

function attribute(node, attributes, type = node.type) {
  const typeKey = type.toLowerCase();
  const map = handlers[typeKey];
  if (!map) throw new Error(`Unknown attribute type: ${typeKey}`);

  let ctx = { node }
  for (const attr of attributes) {
    const [, name] = attr.match(/@(\w+)/i) || [];
    if (!name) continue;
    const handler = map[name.toLowerCase()];
    handler ? handler(node, extractAttributeParams(attr), ctx) : console.warn(`No handler for @${name} on ${typeKey}`);
  }

  return ctx.replace || node(); // Return evaluated block
}

// never wanted to write module.exports or export default again
// @Exports
({
  interpolate,
  template,
  attribute,
})


### module eval

we start by evaluating the code from this notebook and executing our own attributes

attribute code?


In [ ]:
const FUNCNAME = '[a-zA-Z_$][\\w$]*'; // Allow valid JS identifiers
const WORDINESS = '[\\s\\S]*?\\w[\\s\\S]*?'; // Minimal match around at least one word character

// @Nodes
const nodeMatchers = {
  attribute: /@(\w+)\s*(\([^)]*\))?/i,
  function: new RegExp(
    `(?:async\\s+)?function\\s+${FUNCNAME}\\s*\\(|` +         // normal function
    `(?:const|let|var)?\\s*${FUNCNAME}\\s*=\\s*(?:async\\s*)?\\(?([^=]*)\\)?\\s*=>` // arrow function
  ),
  comments: /^\s*\/\/.*/gm, // Better line-matching multiline
  object: new RegExp(`\\(\\{[\\s\\S]*?\\n(\\s*${FUNCNAME}\\s*:\\s*${WORDINESS}\\n)+\\s*\\}\\)`, 'g'),
  logical: new RegExp(`\\(\\s*${WORDINESS}\\s*(?:&&|\\|\\|)\\s*${WORDINESS}\\s*\\)`, 'g'),
  variable: /\b(let|const|var)\s+([a-zA-Z_$][\w$]*)(\s*=\s*[^;]*)?;/g,
  boolean: new RegExp(`\\(\\s*${WORDINESS}\\s*(==|!=|===|!==|>|<|>=|<=)\\s*${WORDINESS}\\s*\\)`, 'g'),
  identifier: /(['"`])(?:\\.|[^\\])*?\1/g, // matches any quoted string with escaped quote support
  assignment: new RegExp(`\\b(${FUNCNAME})\\s*=\\s*(${WORDINESS})`, 'g'),
  params: new RegExp(`function*\\s*${FUNCNAME}\\s*\(([^)]*)\\)|\\(?\\s*([^\)=]+?)\\s*\\)?\\s*=>`, 'g'),
}

// Utility to extract attribute lines and remove them from the code
function extractAttributes(code) {
  const { attributes, remaining } = code.split('\n').reduce((acc, line) => {
    nodeMatchers.attribute.test(line)
      ? acc.attributes.push(line.trim())
      : acc.remaining.push(line);
    return acc;
  }, { attributes: [], remaining: [] });

  return { attributes, code: remaining.join('\n') };
}

// Utility to find the next matching code block after a given index
function findNextNode(code, startIdx) {
  for (const [type, regex] of Object.entries(nodeMatchers)) {
    if (type === 'attribute') continue;

    const match = regex.exec(code.slice(startIdx));
    if (!match) continue;

    const absIdx = startIdx + match.index;
    const [open, close] = ['object', 'function', 'variable', 'assignment'].includes(type) ? ['{', '}'] : ['(', ')'];
    // @Imports(balanced)
    const block = balanced(open, close, code.slice(absIdx));
    
    if (block && block.end !== undefined) {
      const fullBlock = code.slice(absIdx, absIdx + block.end + 1);
      return { type, start: absIdx, end: absIdx + block.end + 1, code: fullBlock };
    }
  }

  return null;
}

// Main parser
function attributeCode(code) {
  let result = '', cursor = 0;

  while (cursor < code.length) {
    const remaining = code.slice(cursor);
    const { attributes, code: stripped } = extractAttributes(remaining);

    if (!attributes.length) {
      result += remaining;
      break;
    }

    const offset = remaining.length - stripped.length;
    const node = findNextNode(remaining, offset);

    if (!node) {
      result += remaining;
      break;
    }

    result += remaining.slice(0, node.start);
    result += `attribute(() => ${node.code}, ${JSON.stringify(attributes)}, "${node.type}")`;
    cursor += node.start + node.code.length;
  }

  return result;
}

module.exports = attributeCode



### basic attributes

by now the attribution system should be loaded and we can provide some basic tools


In [ ]:

// list of aspects implemented in javascript
function inflect(func, ctx) {
  ctx.replace = func
}

// list of helper functions
function prefect(func, ctx) {
  let replace = ctx.replace
  ctx.replace = (...args) => {
    func()
    let node = replace || ctx.node()
    return node(args)
  }
}

function affect(func, ctx) {
  let replace = ctx.replace
  ctx.replace = (...args) => {
    let node = replace || ctx.node()
    let result = node(args)
    func()
    return result
  }
}

// Usage: add stopwatch attribute to any function you want to show how long it took in the logs
// @Attribute
function stopwatch(node, attrs, ctx) {
  let replace = ctx.replace
  inflect((...args) => {
    let func = replace || node()
    let then = Date.now()
    console.log('Running:', func.name, then)
    let result = func(args)
    console.info('Function (' + func.name + ') took:', ((Date.now() - then) / 1000) + 's');
    return result
  }, ctx)
}

// @Attribute
function timer(node, attrs, ctx) {
  // because ctx is per function, this will work for 1 function call per definition
  prefect(() => {
    ctx.then = Date.now()
    console.log('Running:', node().name, ctx.then)
  }, ctx)
  affect(() => {
    console.info('Function (' + node().name + ') took:', ((Date.now() - ctx.then) / 1000) + 's');
  }, ctx)
}

// make this function available to other modules
const interpretCells = (search) => search ? importer.interpret(search)
  : importer.import('cell cache').cellCache.map(([_, cell]) => importer.lookupCell(cell, cacheCells))

// add the cells attribute to any function you want cells inserted into the parameters
// @Attribute
function cells(node, attrs, ctx) {
  timer(node, attrs, ctx)
  let replace = ctx.replace
  inflect((search, ...args) => {
    let func = replace || node()
    let cells = interpretCells(search)
    return func(cells, ...args)
  })
}




### attribute notebook?

finally we are ready to export our newly attributed code cells into a module


In [ ]:
const path = require('path');
const attributeCode = importer.import('attribute code');

const isValidSearch = search => typeof search === 'string' || Array.isArray(search);
const isCellList = cells => cells.language || (cells[0] && cells[0].language);
const loadCells = search => isValidSearch(search)
  ? importer.interpret(search) : isCellList(search)
    ? search : (() => { throw new Error('Input not recognized: ' + search); })();

const getCellCode = cell => cell.source.join('');
const getCellPath = cell => path.join(path.dirname(path.resolve(cell.filename)), cell.id);
const getModule = (code, cell, ctx) => importer.makeModule(code, getCellPath(cell), ctx);

const mergeResult = (mod, result, i) =>
  typeof result === 'object' ? Object.assign(mod, result)
    : typeof result === 'function' ? (mod[result.name] = result)
      : (mod[i] = result);

function attributeNotebook(search) {
  const cells = loadCells(search);
  const module = {};
  const ctx = { attribute: module.attribute };

  for (let i = 0; i < cells.length; i++) {
    const cell = cells[i];
    const code = attributeCode(getCellCode(cell));
    console.log(code)
    const result = getModule(code, cell, ctx);

    if (typeof search === 'string' && cells.length === 1 && !search.endsWith('ipynb')) {
      return result;
    }

    mergeResult(module, result, i);
  }

  return module;
}

module.exports = attributeNotebook('voidzero.ipynb');



### TODO: inject attribution and REPL into chrome extension


## TODO: generic REPL

this will be portable using LLM and/or ANTLR to run through the code and attributes
